In [65]:
import pandas as pd 
import pip._vendor.requests as requests
import spotipy
import numpy as np
import matplotlib as plt
from spotipy.oauth2 import SpotifyOAuth
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.multiclass import OutputCodeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression 
from  sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import pickle
scope1 = "user-library-read"
from spotipy.oauth2 import SpotifyClientCredentials
sp =spotipy.Spotify(auth_manager= SpotifyOAuth(client_id = "15fd015d4da742d0bf7da9c6ed3120c7", client_secret = "349c5181636e4e3cbb974b051db425fd", redirect_uri = "http://localhost", scope= scope1))

Happy song analysis

In [66]:
# df_happy1 = pd.read_csv("happy_songs_everyone_knows__.csv")
# df_happy2 = pd.read_csv("jhjh.csv")
# df_happy_total = pd.concat([df_happy1, df_happy2])
# df_happy_total = df_happy_total.drop(["Spotify ID", "Artist IDs", "Track Name", "Album Name", "Artist Name(s)", "Release Date"], axis = 1)
# df_happy_total["tag"] = "happy"
# df_happy_total.describe()
# # df_happy1[["Danceability","Energy","Speechiness","Acousticness", "Instrumentalness", "Liveness","Valence"]].plot.box(figsize = (10, 10))
# # # a = df_happy1.plot.scatter(x = "Popularity", y= "Energy")
# # # a
# # boxplot = df_happy1.boxplot(column =["Popularity","Danceability","Energy","Speechiness","Acousticness","Instrumentalness", "Liveness"])
# # boxplot
# # plt.show()

In [67]:
df_total = pd.read_csv("total.csv")

In [68]:
# df_total = df_total[~df_total['Tempo'].isin(df_vibe['Tempo'])]

In [69]:
df_total.loc[df_total["tag"] == "calm", "tag"] = 0
df_total.loc[df_total["tag"] == "happy", "tag"] = 1
df_total.loc[df_total["tag"] == "energ", "tag"] = 2
df_total.loc[df_total["tag"] == "sad", "tag"] = 3

In [71]:

# Normalize the DataFrame to be between 0 and 1
min_value = df_total[["Tempo"]].min()
max_value = df_total[["Tempo"]].max()
df_total["Tempo"] = (df_total[["Tempo"]] - min_value) / (max_value - min_value)

min_value = df_total[["Loudness"]].min()
max_value = df_total[["Loudness"]].max()
df_total["Loudness"] = (df_total[["Loudness"]] - min_value) / (max_value - min_value)

min_value = df_total[["Key"]].min()
max_value = df_total[["Key"]].max()
df_total["Key"] = (df_total[["Key"]] - min_value) / (max_value - min_value)



In [74]:
df_total

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,tag
0,0.746,0.754,0.090909,0.861097,0.0676,0.01520,0.000000,0.0486,0.746,0.647064,2
1,0.628,0.593,0.727273,0.899850,0.0323,0.09300,0.000002,0.0960,0.104,0.496042,2
2,0.620,0.805,0.454545,0.888045,0.0417,0.00564,0.000092,0.2850,0.586,0.478033,2
3,0.504,0.785,0.727273,0.905352,0.1040,0.07380,0.000000,0.2390,0.502,0.793408,2
4,0.560,0.872,0.272727,0.917886,0.1400,0.02390,0.000007,0.3710,0.682,0.502263,2
...,...,...,...,...,...,...,...,...,...,...,...
1496,0.688,0.662,0.545455,0.887292,0.0441,0.09710,0.000001,0.1720,0.549,0.435769,1
1497,0.521,0.877,0.636364,0.890255,0.0683,0.01320,0.080600,0.0929,0.199,0.830100,1
1498,0.287,0.473,0.454545,0.934769,0.0340,0.08580,0.000044,0.3830,0.577,0.868232,1
1499,0.673,0.704,0.545455,0.828834,0.3600,0.19600,0.000000,0.0898,0.372,0.264508,1


In [83]:
 df_total[["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]] = df_total[["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"]] 

In [92]:
df_total = df_total.drop(["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"],axis= 1)

In [93]:
data = df_total[["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]] 

In [94]:
data_target = df_total[["tag"]].astype(int)
data_target.dtypes

tag    int64
dtype: object

In [95]:
df_total_after = df_total
df_total_after

,tag,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2,0.746,0.754,0.090909,0.861097,0.0676,0.01520,0.000000,0.0486,0.746,0.647064
1,2,0.628,0.593,0.727273,0.899850,0.0323,0.09300,0.000002,0.0960,0.104,0.496042
2,2,0.620,0.805,0.454545,0.888045,0.0417,0.00564,0.000092,0.2850,0.586,0.478033
3,2,0.504,0.785,0.727273,0.905352,0.1040,0.07380,0.000000,0.2390,0.502,0.793408
4,2,0.560,0.872,0.272727,0.917886,0.1400,0.02390,0.000007,0.3710,0.682,0.502263
...,...,...,...,...,...,...,...,...,...,...,...
1496,1,0.688,0.662,0.545455,0.887292,0.0441,0.09710,0.000001,0.1720,0.549,0.435769
1497,1,0.521,0.877,0.636364,0.890255,0.0683,0.01320,0.080600,0.0929,0.199,0.830100
1498,1,0.287,0.473,0.454545,0.934769,0.0340,0.08580,0.000044,0.3830,0.577,0.868232
1499,1,0.673,0.704,0.545455,0.828834,0.3600,0.19600,0.000000,0.0898,0.372,0.264508


In [ ]:
song_x_train, song_x_test, song_y_train, song_y_test = train_test_split(data, data_target, test_size= 0.2, random_state= 42)
df_result = []


model = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes= (16, 10), activation= "logistic",max_iter= 1000, learning_rate_init= 0.01, alpha = 0.01), n_jobs = -1)
song_y_train = np.ravel(song_y_train)
song_y_test = np.ravel(song_y_test)
model.fit(song_x_train,song_y_train)
# df_total_after["prediciton"] = model.predict(data)
# prediction = np.ravel(df_total_after[["prediciton"]].astype(int))
# output = {"RMSE": np.sqrt(mean_squared_error(tag,prediction)), "accuracy_score":metrics.accuracy_score(tag,prediction)}
# df_result.append(output)
#         # print("RMSE", np.sqrt(mean_squared_error(song_y_test,song_y_predict)))
#         # print(metrics.accuracy_score(song_y_test, song_y_predict))
#         # print("------")
# df_output = pd.DataFrame(df_result)
# df_output
df_total_after
# song_x_test

In [ ]:
df_difference = df_total_after[df_total_after.tag != df_total_after.prediciton]
df_difference[df_difference["tag"] == 2]

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,tag,prediciton
1,0.628,0.593,0.727273,0.899850,0.0323,0.093000,0.000002,0.0960,0.104,0.496042,2,3
6,0.527,0.835,0.545455,0.893689,0.0433,0.016600,0.000000,0.2490,0.654,0.501856,2,1
7,0.572,0.797,0.000000,0.919955,0.0497,0.094900,0.000000,0.2590,0.598,0.561260,2,1
10,0.691,0.602,0.090909,0.828670,0.0497,0.051500,0.000000,0.1410,0.715,0.458827,2,1
12,0.691,0.795,0.090909,0.923717,0.0355,0.000812,0.000001,0.1560,0.876,0.550390,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.645,0.534,0.363636,0.890632,0.0306,0.147000,0.000004,0.0858,0.302,0.283373,2,3
403,0.639,0.698,0.636364,0.854277,0.0702,0.756000,0.000000,0.1460,0.310,0.442823,2,3
405,0.802,0.848,1.000000,0.871091,0.0465,0.011800,0.395000,0.0567,0.165,0.465273,2,3
406,0.545,0.473,1.000000,0.824061,0.0312,0.284000,0.000000,0.0871,0.195,0.526263,2,3


In [ ]:
# mean = df_output["accuracy_score"].mean()
# std = df_output["accuracy_score"].std()
# normal = np.random.normal(mean, std, size = len(df_output))
# plt.hist(normal, bins = 50, density= True)

In [ ]:
# mean = df_output["RMSE"].mean()
# std = df_output["RMSE"].std()
# normal = np.random.normal(mean, std, size = len(df_output))
# plt.hist(normal, bins = 30, density= True)

In [100]:
filename = "model.pkl"
with open (filename, "wb") as file:
    pickle.dump(model, file)


In [ ]:

test_energy = pd.read_csv("engertic.csv")
test_energy

,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,3zkWCteF82vJwv0hRLba76,7n2wHs1TKAczGzO7Dd2rGr,Stitches,Handwritten,Shawn Mendes,2015-04-14,206880,79,spotify:user:,2023-01-27T11:00:00Z,...,1,-6.684,1,0.0676,0.01520,0.000000,0.0486,0.746,149.882,4
1,7xoUc6faLbCqZO6fQEYprd,66CXWjxzNUsdJxJ2JdwvnR,One Last Time,My Everything (Deluxe),Ariana Grande,2014-08-25,197266,82,spotify:user:,2023-01-27T11:00:00Z,...,8,-5.036,1,0.0323,0.09300,0.000002,0.0960,0.104,125.026,4
2,2hHLbkatPwOOmrNxTiD41L,6sFIWsNpZYqfjUpaCgueju,I Really Like You,Emotion (Deluxe Expanded Edition),Carly Rae Jepsen,2015,204763,69,spotify:user:,2023-01-27T11:00:00Z,...,5,-5.538,1,0.0417,0.00564,0.000092,0.2850,0.586,122.062,4
3,1V6gIisPpYqgFeWbMLI0bA,6S2OmqARrzebs0tKUEyXyp,Heart Attack,Demi,Demi Lovato,2013-01-01,210840,76,spotify:user:,2023-01-27T11:00:00Z,...,8,-4.802,1,0.1040,0.07380,0.000000,0.2390,0.502,173.968,4
4,1kPpge9JDLpcj15qgrPbYX,"07QEuhtrNmmZ0zEcqE9SF6,6sFIWsNpZYqfjUpaCgueju",Good Time,The Midsummer Station,"Owl City,Carly Rae Jepsen",2012-01-01,205933,76,spotify:user:,2023-01-27T11:00:00Z,...,3,-4.269,1,0.1400,0.02390,0.000007,0.3710,0.682,126.050,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,6BePGk3eCan4FqaW2X8Qy3,"2o5jDhtHVPhrJdv3cEQ99Z,45dkTj5sMRSjrmBSBeiHym",10:35,10:35,"Tiësto,Tate McRae",2022-11-03,172252,89,spotify:user:,2023-01-27T11:00:00Z,...,8,-5.733,1,0.0970,0.06830,0.000004,0.1800,0.698,120.003,4
146,0U10zFw4GlBacOy9VDGfGL,"5pKCCKE2ajJHZ9KAiaK11H,7CajNmpbOovFoOoasH2HaY",We Found Love,Talk That Talk,"Rihanna,Calvin Harris",2011-11-19,215226,80,spotify:user:,2023-01-27T11:00:00Z,...,1,-4.485,1,0.0383,0.02500,0.001380,0.1080,0.600,127.985,4
147,4zP7ADsgJgHGY6VzxbNp1z,7gOdHgIoIKoe4i9Tta6qdD,Year 3000,Music From Chasing Happiness,Jonas Brothers,2019-05-09,201960,63,spotify:user:,2023-01-27T11:00:00Z,...,11,-5.850,1,0.0437,0.00450,0.000002,0.3350,0.798,106.965,4
148,48QmG1dfvMuYLxMPt7KSRA,"3Isy6kedDrgPYoTS1dazA9,6M2wZ9GZgrQXHCFfjv46we",No Lie,Mad Love The Prequel,"Sean Paul,Dua Lipa",2018-06-29,221176,80,spotify:user:,2023-01-27T11:00:00Z,...,7,-2.862,1,0.1170,0.04660,0.000000,0.2060,0.463,102.040,4


In [ ]:
df = pd.DataFrame()
for i in range (len(test_energy)) :
    song_uri = test_energy.at[i, "Spotify ID"]
    analysis = sp.audio_features(song_uri)
    analy = pd.DataFrame(analysis)
    df = pd.concat([df, analy])
test_energy = df.reset_index().drop("index", axis= 1)

In [ ]:
test_energy

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.746,0.754,1,-6.684,1,0.0676,0.01520,0.000000,0.0486,0.746,149.882,audio_features,3zkWCteF82vJwv0hRLba76,spotify:track:3zkWCteF82vJwv0hRLba76,https://api.spotify.com/v1/tracks/3zkWCteF82vJ...,https://api.spotify.com/v1/audio-analysis/3zkW...,206880,4
1,0.628,0.593,8,-5.036,1,0.0323,0.09300,0.000002,0.0960,0.104,125.026,audio_features,7xoUc6faLbCqZO6fQEYprd,spotify:track:7xoUc6faLbCqZO6fQEYprd,https://api.spotify.com/v1/tracks/7xoUc6faLbCq...,https://api.spotify.com/v1/audio-analysis/7xoU...,197267,4
2,0.620,0.805,5,-5.538,1,0.0417,0.00564,0.000092,0.2850,0.586,122.062,audio_features,2hHLbkatPwOOmrNxTiD41L,spotify:track:2hHLbkatPwOOmrNxTiD41L,https://api.spotify.com/v1/tracks/2hHLbkatPwOO...,https://api.spotify.com/v1/audio-analysis/2hHL...,204764,4
3,0.504,0.785,8,-4.802,1,0.1040,0.07380,0.000000,0.2390,0.502,173.968,audio_features,1V6gIisPpYqgFeWbMLI0bA,spotify:track:1V6gIisPpYqgFeWbMLI0bA,https://api.spotify.com/v1/tracks/1V6gIisPpYqg...,https://api.spotify.com/v1/audio-analysis/1V6g...,210840,4
4,0.560,0.872,3,-4.269,1,0.1400,0.02390,0.000007,0.3710,0.682,126.050,audio_features,1kPpge9JDLpcj15qgrPbYX,spotify:track:1kPpge9JDLpcj15qgrPbYX,https://api.spotify.com/v1/tracks/1kPpge9JDLpc...,https://api.spotify.com/v1/audio-analysis/1kPp...,205933,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.696,0.793,8,-5.733,1,0.0970,0.06830,0.000004,0.1800,0.698,120.003,audio_features,6BePGk3eCan4FqaW2X8Qy3,spotify:track:6BePGk3eCan4FqaW2X8Qy3,https://api.spotify.com/v1/tracks/6BePGk3eCan4...,https://api.spotify.com/v1/audio-analysis/6BeP...,172253,4
146,0.735,0.766,1,-4.485,1,0.0383,0.02500,0.001380,0.1080,0.600,127.985,audio_features,0U10zFw4GlBacOy9VDGfGL,spotify:track:0U10zFw4GlBacOy9VDGfGL,https://api.spotify.com/v1/tracks/0U10zFw4GlBa...,https://api.spotify.com/v1/audio-analysis/0U10...,215227,4
147,0.659,0.857,11,-5.850,1,0.0437,0.00450,0.000002,0.3350,0.798,106.965,audio_features,4zP7ADsgJgHGY6VzxbNp1z,spotify:track:4zP7ADsgJgHGY6VzxbNp1z,https://api.spotify.com/v1/tracks/4zP7ADsgJgHG...,https://api.spotify.com/v1/audio-analysis/4zP7...,201960,4
148,0.742,0.882,7,-2.862,1,0.1170,0.04660,0.000000,0.2060,0.463,102.040,audio_features,48QmG1dfvMuYLxMPt7KSRA,spotify:track:48QmG1dfvMuYLxMPt7KSRA,https://api.spotify.com/v1/tracks/48QmG1dfvMuY...,https://api.spotify.com/v1/audio-analysis/48Qm...,221176,4


In [ ]:
big = ["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"]
small = ["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]

In [ ]:
# test_energy = test_energy[["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]
# Normalize the DataFrame to be between 0 and 1
min_value = test_energy[["Tempo"]].min()
max_value = test_energy[["Tempo"]].max()
test_energy["Tempo"] = (test_energy[["Tempo"]] - min_value) / (max_value - min_value)

min_value = test_energy[["Loudness"]].min()
max_value = test_energy[["Loudness"]].max()
test_energy["Loudness"] = (test_energy[["Loudness"]] - min_value) / (max_value - min_value)

min_value = test_energy[["Key"]].min()
max_value = test_energy[["Key"]].max()
test_energy["Key"] = (test_energy[["Key"]] - min_value) / (max_value - min_value)
test_energy

KeyError: "None of [Index(['Tempo'], dtype='object')] are in the [columns]"

In [ ]:
test_energy[["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"]] = test_energy[["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]] 

In [ ]:
test_energy = test_energy.drop(["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"], axis= 1)

In [ ]:
test_energy["prediciton"] = model.predict(test_energy[["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"]] )

In [ ]:
test_energy

,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,prediciton
0,3zkWCteF82vJwv0hRLba76,7n2wHs1TKAczGzO7Dd2rGr,Stitches,Handwritten,Shawn Mendes,2015-04-14,206880,79,spotify:user:,2023-01-27T11:00:00Z,...,0.247630,1,0.0676,0.01520,0.000000,0.0486,0.746,0.611228,4,2
1,7xoUc6faLbCqZO6fQEYprd,66CXWjxzNUsdJxJ2JdwvnR,One Last Time,My Everything (Deluxe),Ariana Grande,2014-08-25,197266,82,spotify:user:,2023-01-27T11:00:00Z,...,0.521703,1,0.0323,0.09300,0.000002,0.0960,0.104,0.369560,4,3
2,2hHLbkatPwOOmrNxTiD41L,6sFIWsNpZYqfjUpaCgueju,I Really Like You,Emotion (Deluxe Expanded Edition),Carly Rae Jepsen,2015,204763,69,spotify:user:,2023-01-27T11:00:00Z,...,0.438217,1,0.0417,0.00564,0.000092,0.2850,0.586,0.340742,4,2
3,1V6gIisPpYqgFeWbMLI0bA,6S2OmqARrzebs0tKUEyXyp,Heart Attack,Demi,Demi Lovato,2013-01-01,210840,76,spotify:user:,2023-01-27T11:00:00Z,...,0.560619,1,0.1040,0.07380,0.000000,0.2390,0.502,0.845409,4,3
4,1kPpge9JDLpcj15qgrPbYX,"07QEuhtrNmmZ0zEcqE9SF6,6sFIWsNpZYqfjUpaCgueju",Good Time,The Midsummer Station,"Owl City,Carly Rae Jepsen",2012-01-01,205933,76,spotify:user:,2023-01-27T11:00:00Z,...,0.649260,1,0.1400,0.02390,0.000007,0.3710,0.682,0.379516,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,6BePGk3eCan4FqaW2X8Qy3,"2o5jDhtHVPhrJdv3cEQ99Z,45dkTj5sMRSjrmBSBeiHym",10:35,10:35,"Tiësto,Tate McRae",2022-11-03,172252,89,spotify:user:,2023-01-27T11:00:00Z,...,0.405787,1,0.0970,0.06830,0.000004,0.1800,0.698,0.320723,4,2
146,0U10zFw4GlBacOy9VDGfGL,"5pKCCKE2ajJHZ9KAiaK11H,7CajNmpbOovFoOoasH2HaY",We Found Love,Talk That Talk,"Rihanna,Calvin Harris",2011-11-19,215226,80,spotify:user:,2023-01-27T11:00:00Z,...,0.613338,1,0.0383,0.02500,0.001380,0.1080,0.600,0.398330,4,2
147,4zP7ADsgJgHGY6VzxbNp1z,7gOdHgIoIKoe4i9Tta6qdD,Year 3000,Music From Chasing Happiness,Jonas Brothers,2019-05-09,201960,63,spotify:user:,2023-01-27T11:00:00Z,...,0.386330,1,0.0437,0.00450,0.000002,0.3350,0.798,0.193958,4,2
148,48QmG1dfvMuYLxMPt7KSRA,"3Isy6kedDrgPYoTS1dazA9,6M2wZ9GZgrQXHCFfjv46we",No Lie,Mad Love The Prequel,"Sean Paul,Dua Lipa",2018-06-29,221176,80,spotify:user:,2023-01-27T11:00:00Z,...,0.883253,1,0.1170,0.04660,0.000000,0.2060,0.463,0.146074,4,2


In [ ]:
calm = test_energy[test_energy['prediciton'] == 0 ]
happy = test_energy[test_energy['prediciton'] == 1 ]
energy = test_energy[test_energy['prediciton'] == 2 ]
sad = test_energy[test_energy['prediciton'] == 3 ]

In [ ]:
print(len(calm))
calm

0


,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,prediciton


In [ ]:
print(len(happy))
happy

In [ ]:
print(len(energy))
energy

In [ ]:
print(len(sad))
sad

In [ ]:
results = sp.current_user_saved_tracks()


In [ ]:
liked_songs = []
results = sp.current_user_saved_tracks()
while results:
    for item in results['items']:
        track = item['track']
        track_name = track['name']
        artist_name = track['artists'][0]['name']
        album_name = track['album']['name']
        song_id = track['id']
        row = {"track_name" :[track_name], "artist_name" : [artist_name], "album_name" : [album_name], "song_id": [song_id]}
        liked_songs.append(row)
    results = sp.next(results)

df_liked_songs = pd.DataFrame(liked_songs)

In [ ]:
df_liked_songs

,track_name,artist_name,album_name,song_id
0,[踊り子],[Vaundy],[踊り子],[4xk70Qur2QeRmWIzWPC63V]
1,[熱水澡],[Soft Lipa],[收斂水],[6xAtGKQApLCPft6DYR6fFK]
2,[instagram],[DEAN],[instagram],[6z1kLsntE7FuzKZHZWrXYN]
3,[Flowers],[Miley Cyrus],[Endless Summer Vacation],[4DHcnVTT87F0zZhRPYmZ3B]
4,[Joker (Feat. JAMIE)],[BIG Naughty],[Bucket List],[7GLn0324YKGYxbYN7GH8Ip]
...,...,...,...,...
1670,[Story of My Life],[One Direction],[Midnight Memories (Deluxe)],[4nVBt6MZDDP6tRVdQTgxJg]
1671,[Safe And Sound],[Capital Cities],[In A Tidal Wave Of Mystery (Deluxe Edition)],[6BnONjR7itGMEqwxKTIlRM]
1672,[Rolling in the Deep],[Adele],[21],[1eq1wUnLVLg4pdEfx9kajC]
1673,[Someone Like You],[Adele],[21],[6QPKYGnAW9QozVz2dSWqRg]


In [ ]:
like_songs = []
for item in results['items']:
    track = item['track']
    track_name = track['name']